### Imports

In [137]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import time
from transformers import Trainer,TrainingArguments,BartConfig,BartTokenizer,BartModel,BartForConditionalGeneration
from transformers import BartTokenizerFast
from torch.utils import data as data_utils
from torch.utils.data import Dataset, DataLoader

import torch

### Load Data

"big_data" contains precomputed simulations for 198 length sequences.

In [138]:
big_data = "NP_sim_output198_1000000.txt"
big_data_header = ["dna","length","energy","struct","blank","prob"]

"small_data" is only of length 32, with fewer sequences and is more managable

Future work will be on a more flexible model that can handle variable lengths, but current intended use is only on sequences of a known set length.

In [140]:
small_data = "dna_small_sim_output.txt"
small_data_header = ["dna","length","energy","struct"]

Load prefered data set into a pandas dataframe

In [270]:
header_names = small_data_header
data = pd.read_csv(small_data,sep="\\t",header=None,names=header_names)


C:\Users\Shoshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Break the DNA and structure up for the tokenizer 

This is because the tokenizer expects to see words, not individual characters 

(it was made originally for NLP)

(building custom tokenizer may be possible, but model was prebuilt to be used with this tokenizer)

In [271]:
dna = data["dna"]
struct = data["struct"]

dna_list = [" ".join(list(d)) for d in data["dna"]]
struct_list = [" ".join(list(s)) for s in data["struct"]]

In [272]:
length=len(dna[0])

Using same tokenizer model was trained for

In [4]:
tokenizer = BartTokenizerFast.from_pretrained("roberta-base")

### Model and Testing

Load model from trainer output

In [273]:
model = BartForConditionalGeneration.from_pretrained('.\DNA_BART_32', return_dict=True)

This will translate individual sequences or batches of sequences

In [274]:
def translate(dna,model=model,tokenizer=tokenizer,max_length=length+2):#+2 is because models are weird
    # tokenize input
    inputTensor = tokenizer(dna,return_tensors='pt')
    # generate output tensor
    outputTensor = model.generate(inputTensor['input_ids'],
                                 max_length = max_length,
                                  early_stopping=True)
    # decode and clean up output
    output = ["".join(tokenizer.decode(t[2:-1].tolist()).split()) for t in outputTensor]
    return output

In [181]:
inputs = tokenizer(dna_list[:5],return_tensors='pt')

In [ ]:
output = [translate(d) for d in dna_list]

In [269]:
output

['..........................................................................(((((((((((((((((((((((((((........................................)))))))))))))))))))))))))).))...........................']

In [ ]:
with open("bartoutput.txt",'w') as f:
    for l in output:
        f.write(l)
        f.write("\n")